In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datetime
import os
if not os.path.exists("../data/reddit_opinion_PSE_ISR.csv"):
    print("Extract reddit_opinion_PSE_ISR.csv.lzma first")

# Create Custom Dataset
The dataset takes the 7 days of reddit comments before a day, and includes whether there was an event on that day

In [14]:
df = pd.read_csv("../data/reddit_opinion_PSE_ISR.csv")
pd.options.display.max_columns = None
df.iloc[[0]]

/tmp/ipykernel_361993/3990994147.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/reddit_opinion_PSE_ISR.csv")


,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,user_account_created_time,user_awardee_karma,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
0,kdj9wbq,1,An unarmed civilian kid (emphasis on the unarm...,AskMiddleEast,2023-12-15 22:44:56,18jbnmw,Fearless-Low-8565,0,1,0,True,2020-11-25 19:31:17,86.0,0.0,1605.0,7591.0,9282.0,24,NaN,Gaza schools now,1.0,24,0,2023-12-15 22:07:36


In [48]:
dt_converter = lambda date_str: datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") if date_str else None
pd.read_csv("../data/reddit_opinion_PSE_ISR.csv", dtype={
    "comment_id": str,
    "score": np.int32,
    "self_text": str,
    "subreddit": str,
    "controversiality": bool,
    "ups": np.int32,
    # "user_is_verified": bool,
}, converters={
    "created_time": dt_converter,
    "user_account_created_time": dt_converter,
    "post_created_time": dt_converter
}).sort_values("post_created_time")

/tmp/ipykernel_361993/3393760409.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("../data/reddit_opinion_PSE_ISR.csv", dtype={


,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,user_account_created_time,user_awardee_karma,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
463832,jysq2o4,56,Something similar happened in Toronto recently.,ActualPublicFreakouts,2023-09-02 12:02:19,1680ip7,Midnightoclock,False,56,0,True,2018-09-11 22:47:26,3069.0,70.0,13368.0,80185.0,96692.0,597,NaN,Eritrean immigrants (illegals) fighting in the...,0.94,597,0,2023-09-02 11:52:49
463747,jz1was2,5,You too!,ActualPublicFreakouts,2023-09-04 04:44:04,1680ip7,system_deleted,False,5,0,False,NaT,0.0,0.0,0.0,0.0,0.0,597,NaN,Eritrean immigrants (illegals) fighting in the...,0.94,597,0,2023-09-02 11:52:49
463748,jz1w9xn,9,"I won't, you can go to another post. Bye!",ActualPublicFreakouts,2023-09-04 04:43:52,1680ip7,system_deleted,False,9,0,False,NaT,0.0,0.0,0.0,0.0,0.0,597,NaN,Eritrean immigrants (illegals) fighting in the...,0.94,597,0,2023-09-02 11:52:49
463749,jz1w7vo,7,"It seems you watch too much propaganda, Israel...",ActualPublicFreakouts,2023-09-04 04:43:20,1680ip7,system_deleted,False,7,0,False,NaT,0.0,0.0,0.0,0.0,0.0,597,NaN,Eritrean immigrants (illegals) fighting in the...,0.94,597,0,2023-09-02 11:52:49
463750,jz1n753,0,That’s crazy. Israelis have no problem shootin...,ActualPublicFreakouts,2023-09-04 03:27:03,1680ip7,Bigdickasor,True,0,0,True,2022-03-27 10:58:38,16.0,0.0,2875.0,5296.0,8187.0,597,NaN,Eritrean immigrants (illegals) fighting in the...,0.94,597,0,2023-09-02 11:52:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,kdj8j0g,1,This could work. But also the quickest way to ...,AskMiddleEast,2023-12-15 22:36:52,18jbnmw,Fearless-Low-8565,False,1,0,True,2020-11-25 19:31:17,86.0,0.0,1605.0,7591.0,9282.0,24,NaN,Gaza schools now,1.00,24,0,2023-12-15 22:07:36
273,kdj4ap0,1,What do you think that Khamas kid is planning?,AskMiddleEast,2023-12-15 22:12:04,18jbnmw,2chicanerous4u,False,1,0,True,2022-12-01 22:58:35,30.0,0.0,54.0,5372.0,5456.0,24,NaN,Gaza schools now,1.00,24,0,2023-12-15 22:07:36
74,kdj8777,1,They will become smarter by finding radius of ...,AskMiddleEast,2023-12-15 22:34:56,18jbnmw,BigTagda,False,1,0,True,2023-10-16 21:21:32,0.0,0.0,28.0,2335.0,2363.0,24,NaN,Gaza schools now,1.00,24,0,2023-12-15 22:07:36
5,kdj9kxx,1,This is honestly pretty devastating. I can’t i...,Palestine,2023-12-15 22:43:01,18jc3is,Whyeff89,False,1,0,True,2018-10-01 05:27:58,262.0,0.0,45321.0,59702.0,105285.0,4,NaN,IDF accidentally shoots and kills three Israel...,1.00,4,0,2023-12-15 22:27:35


In [47]:
typed_reddit_opinion["post_created_time"].max() - typed_reddit_opinion["post_created_time"].min()


Timedelta('104 days 10:47:12')

In [68]:
class EventDataset(Dataset):
    def __init__(self, acled_csv: str, reddit_opinion_csv: str, transform = None):
        """
        Arguments:
            acled_csv (string): Path to the ACLED csv file.
            reddit_opinion_csv (string): Path to reddit opinion CSV
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.acled = pd.read_csv(acled_csv, converters={
            "event_date": lambda date: datetime.datetime.strptime(date, "%d %B %Y") 
        })
        dt_converter = lambda date_str: datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S") if date_str else None
        self.reddit_opinion = pd.read_csv(reddit_opinion_csv, dtype={
            "comment_id": str,
            "score": np.int32,
            "self_text": str,
            "subreddit": str,
            "controversiality": bool,
            "ups": np.int32,
            # "user_is_verified": bool,
        }, converters={
            "created_time": dt_converter,
            "user_account_created_time": dt_converter,
            "post_created_time": dt_converter
        }).sort_values("post_created_time")
        
        self.transform = transform

    def __len__(self):
        return (self.reddit_opinion["post_created_time"].max() - self.reddit_opinion["post_created_time"].min()).days - 7 # pick days when we can have at least 7 days of comment history


    def __getitem__(self, idx):
        # get all posts from first_comment + idx to first_comment + 7 + idx
        comments_start = (self.reddit_opinion["post_created_time"].min() + datetime.timedelta(days=idx)).replace(hour=0, minute=0, second=0, microsecond=0)
        day = (comments_start + datetime.timedelta(days=7)).replace(hour=0, minute=0, second=0, microsecond=0)
        events_count = self.acled[self.acled["event_date"] == day].shape[0]
        data = {"comments": self.reddit_opinion.loc[(self.reddit_opinion['created_time'] >= comments_start) & (self.reddit_opinion['created_time'] <= day)].values.tolist(),"events": events_count}
        return self.transform(data) if self.transform else data


In [69]:
dataset = EventDataset(acled_csv="../data/acled_middle_east_data.csv", reddit_opinion_csv="../data/reddit_opinion_PSE_ISR.csv")

/tmp/ipykernel_361993/1521915176.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  self.reddit_opinion = pd.read_csv(reddit_opinion_csv, dtype={


In [70]:
dataset[0]

2023-09-09 00:00:00


{'comments': [['jysq2o4',
   56,
   'Something similar happened in Toronto recently.',
   'ActualPublicFreakouts',
   Timestamp('2023-09-02 12:02:19'),
   '1680ip7',
   'Midnightoclock',
   False,
   56,
   0,
   True,
   Timestamp('2018-09-11 22:47:26'),
   3069.0,
   70.0,
   13368.0,
   80185.0,
   96692.0,
   597,
   nan,
   'Eritrean immigrants (illegals) fighting in the streets of Tel Aviv (Israel) - Blues are against Eritrean government and reds are pro :',
   0.94,
   597,
   0,
   Timestamp('2023-09-02 11:52:49')],
  ['jz1was2',
   5,
   'You too!',
   'ActualPublicFreakouts',
   Timestamp('2023-09-04 04:44:04'),
   '1680ip7',
   'system_deleted',
   False,
   5,
   0,
   False,
   NaT,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   597,
   nan,
   'Eritrean immigrants (illegals) fighting in the streets of Tel Aviv (Israel) - Blues are against Eritrean government and reds are pro :',
   0.94,
   597,
   0,
   Timestamp('2023-09-02 11:52:49')],
  ['jz1w9xn',
   9,
   "I won't, you 